### Testing out Distil-Whisper

[GitHub Repo](https://github.com/huggingface/distil-whisper/tree/3c8c15f771139f4c98284486534667a87927ae45) of model.

In [4]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [11]:
display(device)
display(torch_dtype)

'cpu'

torch.float32

In [12]:
# Distil-Whisper model id on hugging face api
model_id = "distil-whisper/distil-large-v2"

# load the model...
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,  # this helps keep loading time as low as possible
    use_safetensors=True  # use safetensors
)

model.to(device)

# ... and the processor
processor = AutoProcessor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

In [14]:
# load audio sample from LibriSpeech corpus
from datasets import load_dataset

dataset = load_dataset('hf-internal-testing/librispeech_asr_dummy', 'clean', split='validation')
sample_audio = dataset[0]['audio']

Extracting data files: 100%|██████████████| 1/1 [00:00<00:00,  1.73it/s]
Generating validation split: 73 examples [00:00, 810.38 examples/s]


In [15]:
result = pipe(sample_audio)
print(result['text'])

 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.


In [16]:
# load local sample audio
green_eggs_and_ham_audio = r"C:\Users\Administrator\Desktop\repos\auto_caption\green_eggs_and_ham_video\green_eggs_and_ham_audio.mp4"
result2 = pipe(green_eggs_and_ham_audio)
print(result2['text'])

 You're listening to a toad-stool in Ferrydusts production of Green Eggs and Ham by Dr. Seuss. Be sure to hit the thumbs-up button and subscribe. It lets YouTube know that you're interested in more videos like this. I am Sam.
